In [0]:
import json
import os
import csv
from typing import Callable

import pandas as pd
import requests
from pyprojroot import here

In [0]:
current_working_directory: str = os.getcwd()
project_dir: str = str(here())
DATA_DIR: str = project_dir + "/data"

print(f"{current_working_directory=}")
print(f"{project_dir=}")
print(f"{DATA_DIR=}")

### Buildging Damage Helper Functions

In [0]:
def convert_json_to_csv(path, output_name):
    json_files = [
        pos_json
        for pos_json in os.listdir(path)
        if pos_json.endswith("post_disaster.json")
    ]

    count = 0
    count1 = 0
    for index, js in enumerate(json_files):
        with open(os.path.join(path, js)) as json_file:
            data_mx_eq = json.load(json_file)
        features = data_mx_eq["features"]

        data_file = open("train_" + output_name + "_geo_details.csv", "a")
        csv_writer = csv.writer(data_file)

        data_file1 = open("train_" + output_name + "_geo_xy_details.csv", "a")
        csv_writer1 = csv.writer(data_file1)

        for lng_lat in features["lng_lat"]:

            if count == 0:
                header = [
                    "feature_type",
                    "damage_level",
                    "uid",
                    "wkt_lt_lg",
                    "lat_lng",
                    "img_name",
                    "sensor",
                    "provider_asset_type",
                    "gsd",
                    "capture_date",
                    "off_nadir_angle",
                    "pan_resolution",
                    "sun_azimuth",
                    "sun_elevation",
                    "target_azimuth",
                    "disaster",
                    "disaster_type",
                    "catalog_id",
                    "original_width",
                    "original_height" "width",
                    "height",
                    "metadata_id",
                ]
                csv_writer.writerow(header)
                print(data_mx_eq["metadata"]["gsd"])
                # print(lng_lat['properties']['feature_type'])
                count += 1

            csv_writer.writerow(
                [
                    lng_lat["properties"]["feature_type"],
                    lng_lat["properties"]["subtype"],
                    lng_lat["properties"]["uid"],
                    lng_lat["wkt"],
                    lng_lat["properties"],
                    data_mx_eq["metadata"]["img_name"],
                    data_mx_eq["metadata"]["sensor"],
                    data_mx_eq["metadata"]["provider_asset_type"],
                    str(data_mx_eq["metadata"]["gsd"]),
                    data_mx_eq["metadata"]["capture_date"],
                    str(data_mx_eq["metadata"]["off_nadir_angle"]),
                    str(data_mx_eq["metadata"]["pan_resolution"]),
                    str(data_mx_eq["metadata"]["sun_azimuth"]),
                    str(data_mx_eq["metadata"]["sun_elevation"]),
                    str(data_mx_eq["metadata"]["target_azimuth"]),
                    data_mx_eq["metadata"]["disaster"],
                    data_mx_eq["metadata"]["disaster_type"],
                    data_mx_eq["metadata"]["catalog_id"],
                    str(data_mx_eq["metadata"]["original_width"]),
                    str(data_mx_eq["metadata"]["original_height"]),
                    str(data_mx_eq["metadata"]["width"]),
                    str(data_mx_eq["metadata"]["height"]),
                    data_mx_eq["metadata"]["id"],
                ]
            )

        for xy in features["xy"]:
            if count1 == 0:
                # print(xy['properties']['feature_type'])
                header1 = ["uid", "wkt", "wkt_xy"]
                csv_writer1.writerow(header1)
                count1 += 1

            csv_writer1.writerow([xy["properties"]["uid"], xy["wkt"], xy["properties"]])

In [0]:
def merge_geo_output_files(output_name):    
        df_f1=pd.read_csv("train_"+output_name+"_geo_details.csv", index_col= False)
        df_f2=pd.read_csv("train_"+output_name+"_geo_xy_details.csv", index_col= False)

        df_f1.uid=df_f1.uid.astype(str)
        df_f2.uid=df_f2.uid.astype(str)

        df_merged = pd.merge(df_f1, df_f2, on='uid')
        df_merged.to_csv('train_'+output_name+'.csv', index=False)

## Data Conversion

### Roof Type Classification

In [0]:
with open(BASE_DIR) as json_file:
    data = json.load(json_file)

features=data["features"]
data_file = open('train_roof_types_geo_details.csv', 'w')
csv_writer = csv.writer(data_file)

count = 0

for feature in features:
    if count == 0:
 
        # Writing headers of CSV file
        header = ["type", "id","roof_material", "verified", "name", "geometry_type", "cordinates", "geometry"]
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file

    csv_writer.writerow([feature["type"],feature["properties"]["id"],feature["properties"]["roof_material"], feature["properties"]["verified"],feature["properties"]["name"], feature["geometry"]["type"], feature["geometry"]["coordinates"], feature["geometry"]] )


data_file.close()

### XView Building Damage Dataset

In [0]:
BDD_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/xview_geotransforms.json"

In [0]:
with open(BDD_BASE_DIR) as json_file:
    bdd_data = json.load(json_file)

In [0]:
keys=bdd_data.keys()

In [0]:
data_file = open('train_roof_damage_geo_details.csv', 'w')
csv_writer = csv.writer(data_file)

count = 0

for key in keys:
    if count == 0:
 
        # Writing headers of CSV file
        header = ["image_id", "location","GEOGCS"]
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file

    csv_writer.writerow([key,bdd_data[key][0],bdd_data[key][1]])


data_file.close()

## Mexico Earthquake - Damage datasets

In [0]:
MX_EQ_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/mexico/post_disaster/"

In [0]:
json_files = [pos_json for pos_json in os.listdir(MX_EQ_BASE_DIR) if pos_json.endswith('.json')]
json_files[:3]

In [0]:
df_f1=pd.read_csv("train_mx_eq_geo_details.csv", index_col= False)
df_f2=pd.read_csv("train_mx_eq_geo_xy_details.csv", index_col= False)

In [0]:
df_f1.uid=df_f1.uid.astype(str)
df_f2.uid=df_f2.uid.astype(str)

In [0]:
df_f1

In [0]:
df_mexico1 = pd.merge(df_f1, df_f2, on='uid')

df_mexico1.head()

In [0]:
df_mexico1.to_csv('train_mexico_eq_blds_damage.csv', index=False)

## Hurricane Florence - damage

In [0]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/florence/post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"huricane_florence")    
    

In [0]:
merge_geo_output_files("huricane_florence")

In [0]:
pd.read_csv('train_huricane_florence.csv').head()

## Hurricane Harvey - damage

In [0]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/harvey/post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"huricane_harvey")   
merge_geo_output_files("huricane_harvey")

## Hurricane Matthew - damage

In [0]:
HC_MW_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/matthew/post_disaster/"
convert_json_to_csv(HC_MW_BASE_DIR,"huricane_matthew")   
merge_geo_output_files("huricane_matthew")

## Guatemala Volcano - damage

In [0]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/guatemala/post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"volcano_guatemala")   
merge_geo_output_files("volcano_guatemala")

## Huricane Micheal - damage

In [0]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/micheal/pre_post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"hurricane_micheal")   
merge_geo_output_files("hurricane_micheal")

## Midwest Flooding - damage

In [0]:
## Guatemala Volcano - damage

HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/midwest_flooding/"
convert_json_to_csv(HC_FL_BASE_DIR,"midwest_flooding")   
merge_geo_output_files("midwest_flooding")

## Palu Tsunami - damage

In [0]:

HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/palu-tsunami/"
convert_json_to_csv(HC_FL_BASE_DIR,"palu_tsunami")   
merge_geo_output_files("palu_tsunami")

## Santa Rosa Wildfire - damage

In [0]:

HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/santa_rosa_wildfire/"
convert_json_to_csv(HC_FL_BASE_DIR,"santa_rosa_wildfire")   
merge_geo_output_files("santa_rosa_wildfire")

## Socal fire - damage

In [0]:

HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/raw/socal-fire/"
convert_json_to_csv(HC_FL_BASE_DIR,"socal-fire")   
merge_geo_output_files("socal-fire")